In [1]:
import pandas as pd

import numpy as np

from elasticsearch.connection import RequestsHttpConnection

from elasticsearch import helpers, Elasticsearch

from elasticsearch_dsl import Search

import csv

In [2]:
# Establishing connection
server="localhost:9200"

es_server= Elasticsearch(server, connection_class=RequestsHttpConnection)

print "Ping elasticsearch:",es_server.ping()

Ping elasticsearch: True


In [3]:
# Creating small indexes
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch("localhost:9200")
# 127.0.0.1:9200, 0.0.0.0:9200

list_of_doc = [{
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
},{
    'author': 'kimchy1',
    'text': 'Elasticsearch: cool. bonsai cool1.',
    'timestamp': datetime.now(),
},
{
    'author': 'kimchy2',
    'text': 'Elasticsearch: cool. bonsai cool2.',
    'timestamp': datetime.now(),
}
]
idx = 0
for doc in list_of_doc:
    idx = idx +1
    res = es.index(index="test-index", doc_type='tweet', id=idx, body=doc)
    print res
    print(res['result'])

# res = es.get(index="test-index", doc_type='tweet', id=1)
# # print res
# print(res['_source'])

# es.indices.refresh(index="test-index")

# res = es.search(index="test-index", body={"query": {"match_all": {}}})
# print res
# print("Got %d Hits:" % res['hits']['total'])
# for hit in res['hits']['hits']:
#     print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

{u'_type': u'tweet', u'_seq_no': 5, u'_shards': {u'successful': 1, u'failed': 0, u'total': 2}, u'_index': u'test-index', u'_version': 6, u'_primary_term': 2, u'result': u'updated', u'_id': u'1'}
updated
{u'_type': u'tweet', u'_seq_no': 2, u'_shards': {u'successful': 1, u'failed': 0, u'total': 2}, u'_index': u'test-index', u'_version': 3, u'_primary_term': 2, u'result': u'updated', u'_id': u'2'}
updated
{u'_type': u'tweet', u'_seq_no': 1, u'_shards': {u'successful': 1, u'failed': 0, u'total': 2}, u'_index': u'test-index', u'_version': 2, u'_primary_term': 2, u'result': u'updated', u'_id': u'3'}
updated


In [4]:
# indexing cvs files into elastic search
from elasticsearch import helpers, Elasticsearch
import csv

es = Elasticsearch()

with open('/Users/aparajita/Desktop/77_cancer_proteomes_CPTAC_itraq.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='my-index', doc_type='my-type')

In [ ]:
import csv
import sys
from elasticsearch import Elasticsearch


def upsert_doc_es(es, index_name, doc_type_name, file_path, identifier_col):
    with open(file_path) as f:
        csv_file_object = csv.reader(f, delimiter=',', quotechar='"')
        # reading the data from the file and capturing the information in the header to use when building our index
        header = csv_file_object.next()
        header = [item.lower() for item in header]
        bulk_data = []  # building up a Python dictionary of data set in a format that the Python ES client can use.
        # here we are loading data by means of bulk indexing
        for row in csv_file_object:
            data_dict = {}
            for i in range(len(row)):
                data_dict[header[i]] = row[i]
            op_dict = {
                "index": {
                    "_index": index_name,
                    "_type": doc_type_name,
                    "_id": data_dict[identifier_col]
                }
            }
            # bulk index request takes 2 lines for each operation, one is metadata and other is actual data.
            # op_dict is metadata
            bulk_data.append(op_dict)
            # data_dict is actual data
            bulk_data.append(data_dict)
    res = es.bulk(index=INDEX_NAME, body=bulk_data, refresh=True)


# create ES client, create index
def create_index(es, INDEX_NAME):
    if es.indices.exists(INDEX_NAME):
        print "Index already exists"
        print("deleting '%s' index..." % (INDEX_NAME))
        res = es.indices.delete(index=INDEX_NAME)
        print(" response: '%s'" % (res))
    # since we are running locally, use one shard and no replicas
    request_body = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        }
    }
    print("creating '%s' index..." % (INDEX_NAME))
    res = es.indices.create(index=INDEX_NAME, body=request_body)
    print(" response: '%s'" % (res))

    print "searching response"
    res = es.search(index=INDEX_NAME, size=10, body={"query": {"match_all": {}}})
    print(" response: '%s'" % (res))


def check_if_indices_exists(es, index_name):
    if es.indices.exists(index_name):
        print "Index exists"
        return True
    else:
        print "Index doesnt exists"
        return False


def search_from(es, index_name, searchFor, searchKey, size):
    return es.search(index=index_name, body={
        "size": size,
        "_source": ["name", "age"],
        'query': {
            'match': {
                searchFor: searchKey,
            }
        }
    })


def create_index_alias(es, index_name, alias_name):
    if check_if_indices_exists(es, index_name):
        alias_status = es.indices.put_alias(index=index_name, name=alias_name, body={
            "filter": {
                "match": {
                    "TId": alias_name
                }
            }
        })
        print "alias created", alias_status
    else:
        print "index doesn't exist, not creating alias"


if __name__ == "__main__":
    host = raw_input("Host for Elastic Search: ")
    port = raw_input("Port no for Elastic Search: ")
    ES_HOST = {"host": host, "port": port}
    # creating ES client
    es = Elasticsearch(hosts=[ES_HOST])
    print "Connected: " + str(es.ping())
    while True:
        print "1. Create index"
        print "2. Upsert data"
        print "3. Query"
        print "4. Exit"
        option = int(raw_input("Enter your option: "))
        if option == 1:
            INDEX_NAME = raw_input("Give index name: ")
            alias = raw_input("Give alias name: ")
            # create an index
            create_index(es, INDEX_NAME)
            create_index_alias(es, INDEX_NAME, alias)

        elif option == 2:
            INDEX_NAME = raw_input("Give index name: ")
            doc_type = raw_input("Give the document type name")
            file_path = raw_input("Give the filepath of the dataset")
            id = raw_input("Give the unique id of the dataset")
            upsert_doc_es(es, INDEX_NAME, doc_type, file_path, id)
            print("Successfully upserted your data...")
        elif option == 3:
            INDEX_NAME = raw_input("Give index name: ")
            field = raw_input("Give search field name to search:")
            search_key = raw_input("Give search key :")
            size = int(raw_input("Give result size:"))
            search_val = search_from(es, INDEX_NAME, field, search_key, size)
            for hit in search_val['hits']['hits']:
                print(hit['_source']['name'])
        elif option == 4:
            print("Bye! Bye! Exiting.......")
            sys.exit(0)
        else:
            print("Invalid option, please select correct option")

Host for Elastic Search: http://127.0.0.1:9200/
Port no for Elastic Search: 1
Connected: False
1. Create index
2. Upsert data
3. Query
4. Exit
Enter your option: 1
